In [1]:
%%capture
!pip install datasets
!pip install tokenizers
!pip install torchmetrics
!pip install wandb

In [2]:

%%capture
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Mestrado/ML/topicos_avançados/from0')
import os

# Substitua 'caminho/novo/diretorio' pelo caminho do diretório desejado
os.chdir('/content/drive/MyDrive/Mestrado/ML/topicos_avançados/from0')


In [3]:
import config
import torch
import torch.nn as nn
import pathlib
import torch.optim as optim
import train
import transformer
import pandas as pd


cfg = config.get_config()
print(cfg)

{'batch_size': 16, 'num_epochs': 20, 'lr': 1e-05, 'seq_len': 200, 'd_model': 256, 'num_heads': 8, 'num_of_blocks': 8, 'dff': 1024, 'dropout': 0.1, 'dataset': 'click_bait', 'model_folder': 'weights', 'model_basename': 'tmodel_', 'preload': 'latest', 'tokenizer_file': 'tokenizer.json', 'experiment_name': 'runs/tmodel', 'num_class': 2, 'dataset_path': 'data\\final_data.csv'}


In [4]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
cfg['model_folder'] = 'weights'
cfg['tokenizer_file'] = '/content/drive/MyDrive/Mestrado/ML/topicos_avançados/from0/tokenizer.json'
cfg["experiment_name"] =  '/content/drive/MyDrive/Mestrado/ML/topicos_avançados/from0/runs/tmodel'
cfg['dataset_path'] = "/content/drive/MyDrive/Mestrado/ML/topicos_avançados/from0/data/final_data.csv"

In [7]:
train_dataloader, val_dataloader, tokenizer = train.get_ds(cfg)
model = train.get_model(cfg, tokenizer.get_vocab_size()).to(device)

Max length of source sentence: 145
Number of weights: 9,619,458


In [8]:
saves = torch.load('/content/drive/MyDrive/Mestrado/ML/topicos_avançados/from0/click_bait_weights/config1_v2/tmodel_00.pt',map_location=torch.device(device))
model.load_state_dict(saves['model_state_dict'])

<All keys matched successfully>

In [9]:
print(len(val_dataloader))

6369


In [10]:
criterion = nn.CrossEntropyLoss()  # Substitua pelo seu critério de perda
optimizer = torch.optim.Adam(model.parameters(), lr=cfg['lr'], eps=1e-9)  # Substitua pelo seu otimizador

def calcula_metricas(model,val_dataloader):
  # Coloque o modelo em modo de avaliação (evaluation mode)
  model.eval()

  # Desative o cálculo dos gradientes para acelerar o processo
  with torch.no_grad():
      validation_loss = 0.0
      count=0
      # Loop através dos dados de validação
      for data in val_dataloader:  # Substitua pelo seu dataloader de validação
          # Forward pass
          outputs = model(data['encoder_input'].to(device),data['encoder_mask'].to(device)).to(device)
          labels = data['label'].to(device)
          # Calcular a perda
          if torch.argmax(torch.softmax(outputs, dim=-1), dim=-1) == labels:
            count+=1
          loss = criterion(outputs, labels).to(device)

          # Acumular a perda total
          validation_loss += loss.item() * data['encoder_input'].size(0)  # Multiplicar pela quantidade de amostras no batch

      # Calcular a perda média
      validation_loss = validation_loss / val_dataloader.__len__()  # Dividir pela quantidade total de amostras
      acc = count/val_dataloader.__len__()
  return validation_loss,acc

In [ ]:
print(calcula_metricas(model,val_dataloader))

(0.5446382765353316, 0.7514523473072696)


In [11]:
validation_losses = []
accuracy = []

for epoch in range(10):
  saves = torch.load(f'/content/drive/MyDrive/Mestrado/ML/topicos_avançados/from0/click_bait_weights/config1_v2/tmodel_0{epoch}.pt',map_location=torch.device(device))
  model.load_state_dict(saves['model_state_dict'])
  loss, acc = calcula_metricas(model,val_dataloader)
  validation_losses.append(loss)
  accuracy.append(acc)
print(validation_losses)
print(accuracy)



[0.11578730968479672, 0.08396180580228609, 0.060230473146810085, 0.06580144686742308, 0.05779609747651789, 0.04451363878843568, 0.029471158286464114, 0.025823679097675675, 0.02200312198159391, 0.025401490630007198]
[0.9594912859161564, 0.972994190610771, 0.9814727586748312, 0.97817553776103, 0.9821008007536505, 0.9849269901083373, 0.9915214319359397, 0.9918354529753494, 0.9940336002512168, 0.9945046318103313]


In [12]:
df = pd.DataFrame({'Loss': validation_losses,'Acc': accuracy,})

# Caminho do arquivo CSV
caminho_arquivo = 'metricas_config1_v2.csv'

# Salvar o DataFrame como um arquivo CSV
df.to_csv(caminho_arquivo, index=False)